In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import matplotlib.pyplot as plt

class DataGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Data Viewer and Analyzer")
        self.root.geometry("800x600")

        self.data = None 

        tk.Button(root, text="Load CSV", command=self.load_csv, bg="blue", fg="white", font=("Arial", 12)).pack(pady=10)
        tk.Button(root, text="Sort Data", command=self.sort_data, bg="orange", fg="white", font=("Arial", 12)).pack(pady=10)
        tk.Button(root, text="Show Visualizations", command=self.show_visualizations, bg="green", fg="white", font=("Arial", 12)).pack(pady=10)

        self.tree = ttk.Treeview(
            root, columns=("Date", "Instagram", "YouTube", "WhatsApp"), show='headings'
        )
        for col in self.tree["columns"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=150)
        self.tree.pack(fill=tk.BOTH, expand=True, pady=10)

    def load_csv(self):
        # Load and clean the CSV data
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            try:
                self.data = pd.read_csv(file_path, header=1)
                self.data.columns = ['Date', 'Instagram', 'YouTube', 'WhatsApp']
                self.data.dropna(inplace=True)
                for col in ['Instagram', 'YouTube', 'WhatsApp']:
                    self.data[col] = self.data[col].str.replace(' mb', '').astype(float)
                self.data['Date'] = pd.to_datetime(self.data['Date'], format='%d-%m-%Y')
                self.display_data(self.data)
                messagebox.showinfo("Success", "Data loaded successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load and clean data: {e}")

    def display_data(self, data):
        for row in self.tree.get_children():
            self.tree.delete(row)
        for _, row in data.iterrows():
            self.tree.insert("", "end", values=tuple(row))

    def sort_data(self):
        if self.data is not None:
            try:
                sorted_data = self.data.sort_values(by="Instagram", ascending=False)
                self.display_data(sorted_data)
                messagebox.showinfo("Success", "Data sorted by Instagram usage!")
            except KeyError:
                messagebox.showerror("Error", "Required column for sorting not found!")
        else:
            messagebox.showwarning("Warning", "No data to sort! Load a file first.")

    def show_visualizations(self):
        if self.data is not None:
            try:
                plt.figure(figsize=(10, 6))
                plt.bar(self.data['Date'], self.data['Instagram'], label='Instagram', color='blue')
                plt.bar(self.data['Date'], self.data['YouTube'], bottom=self.data['Instagram'], label='YouTube', color='red')
                plt.bar(self.data['Date'], self.data['WhatsApp'], bottom=self.data['Instagram'] + self.data['YouTube'], label='WhatsApp', color='green')
                plt.title("Daily Internet Usage (Stacked Bar Chart)")
                plt.xlabel("Date")
                plt.ylabel("Usage (MB)")
                plt.xticks(rotation=45)
                plt.legend()
                plt.tight_layout()
                plt.show()

                total_usage = self.data[['Instagram', 'YouTube', 'WhatsApp']].sum()
                plt.figure(figsize=(6, 6))
                plt.pie(total_usage, labels=total_usage.index, autopct='%1.1f%%', colors=['blue', 'red', 'green'])
                plt.title("Total Usage by App")
                plt.tight_layout()
                plt.show()

            except Exception as e:
                messagebox.showerror("Error", f"Visualization error: {e}")
        else:
            messagebox.showwarning("Warning", "No data to visualize! Load a file first.")

if __name__ == "__main__":
    root = tk.Tk()
    app = DataGUI(root)
    root.mainloop()
